In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt

labelName = ["bowl", "dog", "feel", "get", "I", "know", "must", "sick", "you", "zero"]
batchSize = 128
trainingRatio = 0.8
device = torch.device("cuda")

count = 0
rawDataSets = []
for name in labelName:
    data = np.load("data/%s.npy" %(name)) # Dimension M * 11, (第一组bowl,M=2666)
    label = np.zeros((data.shape[0], 1), dtype=np.int32) # Dimension M * 1
    label += count
    data = np.concatenate((data, label), axis=-1) # Dimension M * 12
    if count == 0:
        rawDataSets = data.copy()
    else:
        rawDataSets = np.concatenate((rawDataSets, data), axis=0)
    count += 1

# rawDataSets Dimension 21447 * 12
np.random.shuffle(rawDataSets)

totalCount = rawDataSets.shape[0] 
trainingCount = int(totalCount * trainingRatio) # must be 21447 ,rawDataSets.shape[1], must be 12
print(trainingCount)

# np.random.shuffle之后，就是按照比例，前80% 是training,后20%是validating
trainingData = torch.from_numpy(rawDataSets[0:trainingCount, 0:-1]).to(dtype=torch.float32).to(device)
trainingLabel = torch.from_numpy(rawDataSets[0:trainingCount, -1]).to(dtype=torch.long).to(device)
validatingData = torch.from_numpy(rawDataSets[trainingCount:, 0:-1]).to(dtype=torch.float32).to(device)
validatingLabel = torch.from_numpy(rawDataSets[trainingCount:, -1]).to(dtype=torch.long).to(device)

trainingDataSet = TensorDataset(trainingData, trainingLabel)
trainingDateLoader = DataLoader(trainingDataSet, batch_size=batchSize, shuffle=True)

validatingDataSet = TensorDataset(validatingData, validatingLabel)
validatingDataLoader = DataLoader(validatingDataSet, batch_size=batchSize, shuffle=True)


class FC(nn.Module):
    def __init__(self):
        super(FC, self).__init__()
        self.hidden1=nn.Sequential(
                nn.Linear(in_features=11,out_features=256,bias=True), # 20+5 / 20+11
                nn.Dropout(p = 0.2),
                nn.ReLU())
        self.hidden2=nn.Sequential(
                nn.Linear(in_features=256,out_features=128,bias=True),
                nn.ReLU())
        self.hidden3=nn.Sequential(
                nn.Linear(in_features=128,out_features=100,bias=True),
                nn.ReLU())
        self.hidden4=nn.Sequential(
                nn.Linear(in_features=100,out_features=10,bias=True))

    def forward(self,x):
        x=self.hidden1(x)
        x=self.hidden2(x)
        x=self.hidden3(x)
        output=self.hidden4(x)
        return output
    
net = FC()
net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)

iterr=[]
for i in range(100):
    iterr.append(i)

acc_val=[]

for epoch in range(100):
    statisticLoss = 0.0
    total = 0
    correct = 0
    # Training
    for i, (features, labels) in enumerate(trainingDateLoader):
        optimizer.zero_grad()
        outputs = net(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        statisticLoss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    trainingAcc = 100 * correct / total
    
    # Validating
    total = 0
    correct = 0
    with torch.no_grad():
        for i, (features, labels) in enumerate(validatingDataLoader):
            outputs = net(features)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0) #128
            correct += (predicted == labels).sum().item()
    validatingAcc = 100 * correct / total
    acc_val.append(validatingAcc)

    print("Epoch %d, Average Training Loss: %.4lf, Training Accuracy: %.4lf%%, Validating Accuracy: %.4lf%%" %(epoch+1, statisticLoss, trainingAcc, validatingAcc))

torch.save(net, "/root/moving_sensor_new.pth")

'''
plt.figure(1)
plt.plot(iterr,acc_val)
plt.title('Validating accuracy of sensor_part')
plt.xlabel('epoch')
plt.ylabel('training accuracy')
plt.savefig('moving_sensor.png')
plt.show()
'''

21157
Epoch 1, Average Training Loss: 311.0402, Training Accuracy: 40.3318%, Validating Accuracy: 54.0643%
Epoch 2, Average Training Loss: 186.0586, Training Accuracy: 62.5656%, Validating Accuracy: 73.4783%
Epoch 3, Average Training Loss: 128.6436, Training Accuracy: 75.9229%, Validating Accuracy: 80.1701%
Epoch 4, Average Training Loss: 97.8851, Training Accuracy: 81.1930%, Validating Accuracy: 83.2892%
Epoch 5, Average Training Loss: 80.7149, Training Accuracy: 84.1849%, Validating Accuracy: 85.6900%
Epoch 6, Average Training Loss: 68.8453, Training Accuracy: 86.5765%, Validating Accuracy: 87.6560%
Epoch 7, Average Training Loss: 59.8822, Training Accuracy: 88.4766%, Validating Accuracy: 88.9036%
Epoch 8, Average Training Loss: 53.8653, Training Accuracy: 89.3558%, Validating Accuracy: 89.6408%
Epoch 9, Average Training Loss: 49.0729, Training Accuracy: 90.1735%, Validating Accuracy: 90.7183%
Epoch 10, Average Training Loss: 45.0232, Training Accuracy: 90.8919%, Validating Accuracy:

"\nplt.figure(1)\nplt.plot(iterr,acc_val)\nplt.title('Validating accuracy of sensor_part')\nplt.xlabel('epoch')\nplt.ylabel('training accuracy')\nplt.savefig('moving_sensor.png')\nplt.show()\n"

In [8]:
a=np.load("data/zero.npy")
print(a.shape)
print(a[0])

(2617, 11)
[-0.8815332   0.13792245 -0.35283038 -0.34682727 -0.63621396  0.86147
  0.9255241  -1.3825581   1.2934408  -0.8361659   1.2989041 ]
